<a href="https://colab.research.google.com/github/ayushmangupta/TF2/blob/master/TF2MnistFashionBasicmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip3 install tensorflow-gpu==2.0.0-beta1
#!pip3 install tensorflow_datasets
%matplotlib inline
%load_ext tensorboard
import os
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as tfk
import numpy as np
import tensorflow.keras.layers as tfkl
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
print(tf.__version__)
log_dir = "logd"
!rm -rf logd/

2.0.0-beta1


In [0]:
class Model(tfk.Model):
  def __init__(self):
    super(Model,self).__init__()
    self.common_layers=[]
    self.common_layers.append(tfkl.Conv2D(32,3,strides=(1,1),padding='same',activation="relu",input_shape=(28,28,1)))
    self.common_layers.append(tfkl.BatchNormalization())
    self.common_layers.append(tfkl.MaxPool2D((2,2),(2,2)))
    self.common_layers.append(tfkl.Conv2D(32,3,strides=(1,1),padding='same',activation="relu"))
    self.common_layers.append(tfkl.BatchNormalization())
    self.common_layers.append(tfkl.MaxPool2D((2,2),(2,2)))
    self.common_layers.append(tfkl.Dropout(0.2))
    self.common_layers.append(tfkl.Conv2D(64,3,strides=(1,1),padding='same',activation="relu"))
    self.common_layers.append(tfkl.BatchNormalization())
    self.common_layers.append(tfkl.MaxPool2D((2,2),(2,2)))
    self.common_layers.append(tfkl.Conv2D(64,3,strides=(1,1),padding='same',activation="relu"))
    self.common_layers.append(tfkl.BatchNormalization())
    self.common_layers.append(tfkl.MaxPool2D((2,2),(2,2)))
    self.common_layers.append(tfkl.Dropout(0.2))
    self.common_layers.append(tfkl.Flatten())
    self.common_layers.append(tfkl.Dense(512,activation="relu"))
    self.common_layers.append(tfkl.Dropout(0.2))
    self.common_layers.append(tfkl.Dense(32,activation="relu"))
    self.common_layers.append(tfkl.Dense(10,activation="softmax"))
                              
  def call(self,x):
    x= tf.convert_to_tensor(x,dtype=tf.float32)
    for layer in self.common_layers:
      x = layer(x)
    return x


In [0]:
def plot_filters(model,ii):
    d = model.layers[0]
    o = d(ii)
    mainImg=[]
    for i in range(8): 
        a = []
        for j in range(8):
            a.append(o[i,:,:,j])
        imgg = np.hstack(a)
        mainImg.append(imgg)
    return np.vstack(mainImg)

In [4]:
mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = tf.cast(x_train,tf.float32)
x_test = tf.cast(x_test,tf.float32)


# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
ii = x_train[:10,:,:,:]

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(0.001)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [0]:
model = Model()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
test_log_dir = 'logd/gradient_tape/' + current_time + '/test'

In [0]:
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [12]:
EPOCHS = 15
#tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    

    for test_images, test_labels in test_ds:
      test_step(test_images, test_labels)

    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
        filter_img = plot_filters(model,ii)
        filter_img = tf.summary.image('image',filter_img[np.newaxis,:,:,np.newaxis],step=epoch)
        #tf.summary.trace_export(name="my_func_trace", step=0, profiler_outdir=train_log_dir)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5259063839912415, Accuracy: 80.375, Test Loss: 0.3490198254585266, Test Accuracy: 86.52999877929688
Epoch 2, Loss: 0.4142681956291199, Accuracy: 84.57333374023438, Test Loss: 0.30363813042640686, Test Accuracy: 88.7300033569336
Epoch 3, Loss: 0.3610939681529999, Accuracy: 86.55611419677734, Test Loss: 0.30649441480636597, Test Accuracy: 89.31000518798828
Epoch 4, Loss: 0.3269927501678467, Accuracy: 87.83333587646484, Test Loss: 0.31819021701812744, Test Accuracy: 88.94000244140625
Epoch 5, Loss: 0.30234432220458984, Accuracy: 88.7509994506836, Test Loss: 0.33387187123298645, Test Accuracy: 88.94000244140625
Epoch 6, Loss: 0.28294822573661804, Accuracy: 89.46694946289062, Test Loss: 0.31026577949523926, Test Accuracy: 90.09000396728516
Epoch 7, Loss: 0.2667304575443268, Accuracy: 90.07714080810547, Test Loss: 0.2997649312019348, Test Accuracy: 90.4000015258789
Epoch 8, Loss: 0.2533303499221802, Accuracy: 90.58125305175781, Test Loss: 0.3071303963661194, Test Accuracy: 9